In [ ]:
# Imports
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append('/Users/k4robins/Devel/0_Thesis/housing_app')
from model.model import City
from model.agents import Land, Person, Firm, Bank, Investor, Realtor, Bid, Allocation

In [ ]:
# Parameters
num_steps  = 40
parameters = {
            'run_notes': 'Debugging model.',
            'subfolder': None,
            'width': 50,
            'height':4,
            # LABOUR MARKET AND FIRM PARAMETERS
            'subsistence_wage': 40000.,  # psi
            'init_city_extent': 10.,  #  CUT OR CHANGE?
            'seed_population': 400,
            'init_wage_premium_ratio': 0.2, # 1.2, ###
            #
            # PARAMETERS MOST LIKELY TO AFFECT SCALE
            'c': 300.0,                            ###
            'price_of_output': 10,                 ######
            'density':600,                         #####
            'A': 3000,                             ### 
            'alpha': 0.18,
            'beta':  0.75,
            'gamma': 0.12, ### reduced from .14
            'overhead': 1,
            'mult': 1.2,
            'adjN': 0.15,
            'adjk': 0.05,
            'adjn': 0.25,
            'adjF': 0.15,
            'adjw': 0.15, 
            #'dist': 1, 
            'init_agglomeration_population': 100000.0,
            'init_F': 100.0,
            'init_k': 100.0,
            'init_n': 100.0,
            # HOUSING AND MORTGAGE MARKET PARAMETERS
            'mortgage_period': 5.0,       # T, in years
            'working_periods': 40,        # in years
            'savings_rate': 0.3,
            'discount_rate': 0.07,        # 1/delta
            'r_margin': 0.01,
            'property_tax_rate': 0.04,     # tau, annual rate, was c
            'housing_services_share': 0.3, # a
            'maintenance_share': 0.2,      # b
            'max_mortgage_share': 0.9,
            'ability_to_carry_mortgage': 0.28,
            'wealth_sensitivity': 0.1,
        }



In [ ]:
# Model
model = City(num_steps, **parameters)

# Done
DONE I actually want to use the people and space initialized in the model but make an alternate step function...First normal step function
DONE adjust the step function to test selling homes/bids as bellow
DONE  there is no buyer - why? fix sell_homes to complete the sale as specified

NOW get assertions working
make sure agents and bank both use the offical add_bid method to add bids (maybe see if we can inspect their bids)
Check dedicated investor agent
Remove extra
Access willingnes to pay and compute final price

In [ ]:
def test_sell_homes():
    realtor = Realtor(1, model, (0,0))
    
    # Get bidders and properties
    bidders = [agent for agent in model.schedule.agents if isinstance(agent, Person)]  #[:4]
    properties = [agent for agent in model.schedule.agents if isinstance(agent, Land)] #[:2]

    # Set example bids for the selected agents
    realtor.add_bid(bidders[0], properties[0], 1000)
    realtor.add_bid(bidders[1], properties[0], 150)
    realtor.add_bid(bidders[2], properties[1], 200)
    realtor.add_bid(bidders[3], properties[1], 180)
    realtor.add_bid(bidders[4], properties[0], 15)
    realtor.add_bid(bidders[5], properties[1], 1500)
    realtor.add_bid(bidders[5], properties[2], 1300)
    # Sell homes and get the allocations
    allocations = realtor.sell_homes()
    
    # Check the allocations
    print("len allocations: ", len(allocations))

    # Check the allocations
    assert len(allocations) == 3

    assert allocations[0].property           == properties[0] # "Property1"
    assert allocations[0].successful_bidder  == bidders[0] # "Bidder2"
    assert allocations[0].highest_bid        == 1000
    assert allocations[0].second_highest_bid == 150
    # assert allocations[0].final_price        == 1000

    assert allocations[1].property           == properties[1] # "Property2"
    assert allocations[1].successful_bidder  == bidders[5] # "Bidder3"
    assert allocations[1].highest_bid        == 1500
    assert allocations[1].second_highest_bid == 200
    # assert allocations[1].final_price == 1500

    assert allocations[2].property           == properties[2] # "Property2"
    assert allocations[2].successful_bidder  == bidders[5] # "Bidder3"
    assert allocations[2].highest_bid        == 1300
    assert allocations[2].second_highest_bid == 0
    assert allocations[2].final_price == 1300



    # assert len(allocations) == 2

    # allocation1 = allocations[0]
    # print(f"Allocation 1: Property: {allocation1.property}, Successful Bidder: {allocation1.successful_bidder}, Bid Price: {allocation1.bid_price}, Final Price: {allocation1.final_price}")
    # assert allocation1.property == "Property  1"
    # assert allocation1.successful_bidder == "Bidder2"
    # assert allocation1.bid_price == 150
    # assert allocation1.final_price == 100
    
    # allocation2 = allocations[1]
    # print(f"Allocation 2: Property: {allocation2.property}, Successful Bidder: {allocation2.successful_bidder}, Bid Price: {allocation2.bid_price}, Final Price: {allocation2.final_price}")
    # assert allocation2.property == "Property2"
    # assert allocation2.successful_bidder == "Bidder3"
    # assert allocation2.bid_price == 200
    # assert allocation2.final_price == 180

test_sell_homes()

In [ ]:
# # The normal city step function for testing
# def model_step():
#     model.time_step += 1

#     # logger.debug(f'Step {model.schedule.steps}.')
#     model.step_price_data.clear()

#     # Land records locational rents and calculates price forecast
#     model.schedule.step_breed(Land)
#     new_df = pd.DataFrame(model.step_price_data)
#     model.price_data = pd.concat([model.price_data, new_df], 
#                                         ignore_index=True)

#     model.price_model = model.get_price_model()
#     model.p_dot       = model.get_p_dot()

#     # Firms update wages
#     model.schedule.step_breed(Firm)

#     # People work, retire, and list homes to sell
#     model.schedule.step_breed(Person)

#     for i in model.workforce.retiring:
#         # Add agents to replace retiring workers
#         person = model.create_newcomer()
#         # if (person.unique_id in model.workforce.newcomers):
#         #     print('new agent in newcomers')
#         # else:
#         #     print('new agent not in newcomers')
#         person.bid()

#     # Investors bid on properties
#     model.schedule.step_breed(Investor, step_name='bid')

#     # Realtors sell homes
#     model.schedule.step_breed(Realtor, step_name='sell_homes')

#     # Realtors rent properties
#     model.schedule.step_breed(Realtor, step_name='rent_homes')

#     # Advance model time
#     model.schedule.step_time()

#     model.record_step_data()
# bidding_step()
